### Welcome!

### Flow control

* EAFP - as opposed to LBYL

How to convert a string to a float?

* use regular expressions

In [1]:
user_input = "hello 1_200e-2"

In [2]:
for token in user_input.split():
    try:
        print(float(token))
    except ValueError:
        pass

12.0


* dictionary access often uses EAFP

In [3]:
data = {"a": 123}
try:
    v = data["b"]
except KeyError as exc:
    # do something exc, re-raise, ...
    v = "some default"
print(v)

some default


In [4]:
data.get("b", "some default") # collections ...

'some default'

In [5]:
found = False
for i in range(1, 10, 3):
    if i % 3 == 0:
        found = True
        break

if not found:
    print("no number divisible by 3 found")

no number divisible by 3 found


* use any to check for predicate

In [6]:
for i in range(1, 10, 3):
    if i % 3 == 0:
        break
else:
    print("no number divisible by 3 found")

no number divisible by 3 found


In [7]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Match statement


* match statement -- PEP 636 "structural pattern matching"

In [8]:
!python --version

Python 3.11.0


In [9]:
match None:
    case _:
        pass

In [10]:
def status_text(status=200):
    match status:
        case 200:
            return "OK"
        case 300:
            return "Multiple Choices"
        
    raise ValueError(f"invalid {status}")

In [11]:
status_text(300)

'Multiple Choices'

Alternative values

In [12]:
def status_text(status=200):
    match status:
        case 200:
            return "OK"
        case 401 | 472:
            return "Auth error"
        case 300:
            return "Multiple Choices"
        
    raise ValueError(f"invalid {status}")

In [13]:
status_text(472)

'Auth error'

Text adventure

In [14]:
def parse_command(s):
    match s.split():
        case [command]:
            print(f"ok, executing {command}")
        case ["go", direction]:
            print(f"going {direction}")
        case _:
            print("unknown command")

In [15]:
parse_command("drop shield")

unknown command


In [16]:
parse_command("go north")

going north


In [17]:
parse_command("walk")

ok, executing walk


Subpatterns



In [18]:
def parse_command(s):
    match s.split():
        case [command]:
            print(f"ok, executing {command}")
        case ["go", direction]:
            print(f"going {direction}")
        case ["drop", *items]:
            print(f"dropping {len(items)} items: {items}")
        case _:
            print("unknown command")

In [19]:
parse_command("drop shield axe")

dropping 2 items: ['shield', 'axe']


Subpatterns

In [20]:
def parse_command(s):
    match s.split():
        case [command]:
            print(f"ok, executing {command}")
        case ["go", ("north" | "south") as direction]:
            print(f"going {direction}")
        case ["drop", *items]:
            print(f"dropping {len(items)} items: {items}")
        case _:
            print("unknown command")

In [21]:
parse_command("go west")

unknown command


In [22]:
parse_command("go south")

going south


In [23]:
valid_directions = set(["north", "south", "west", "east"])

In [24]:
def parse_command(s):
    match s.split():
        case [command]:
            print(f"ok, executing {command}")
        case ["go", direction] if direction in valid_directions:
            print(f"going {direction}")
        case ["drop", *items]:
            print(f"dropping {len(items)} items: {items}")
        case _:
            print("unknown command")

In [25]:
parse_command("go west")

going west


Matching object

In [26]:
import datetime

In [27]:
def action_for_date(date):
    match date:
        case datetime.date(year=2000):
            print("archive")
        case datetime.date(year=2384):
            print("join starfleet")
        case _:
            print("go out and play")

In [28]:
action_for_date(datetime.date(2100, 1, 1))

go out and play


Matching dictionaries

In [29]:
def validate(data):
    match data:
        case {"a": 100, "b": _}:
            print("ok")
        case {"a": 50, "b": "err"}:
            print("critical")
        case {"a": 0}:
            print("defunkt")
        case {"a": value, "b": "ok"} if 100 < value < 110:
            print("elevated")

In [30]:
validate({"a": 107, "b": "ok"})

elevated


In [31]:
validate({"a": 100, "b": "abc"})

ok


In [32]:
validate({"a": 50, "b": "err"})

critical


In [33]:
validate({"a": 0, "b": "abc"})

defunkt


## Data Model

* internal of python
* special methods that python uses for its facilities (magic methods)

Categories:

* Initialization
* Comparison
* Unary operators
* Augmented assignment
* Type conversions
* String Magic
* Attribute Magic
* Operator Magic

### Initialization

In [34]:
class A:
    pass

In [35]:
a = A()

In [36]:
a.x = 1

In [37]:
a.x

1

In [38]:
class A:
    def __init__(self):
        self.x = 1

In [39]:
a = A()

In [40]:
a.x

1

In [41]:
class Counted:
    count = 0
    def __init__(self):
        Counted.count += 1
        self.x = 1

In [42]:
a = Counted()
b = Counted()

In [43]:
b.count

2

Finalizer

In [44]:
class A:
    def __del__(self):
        print("__del__")

In [45]:
x = [A()]

In [46]:
x = None

__del__


Class creation

In [47]:
class A:
    def __new__(cls):
        print(f"__new__ {cls}")
    def __init__(self):
        print("__init__")

In [48]:
a = A()

__new__ <class '__main__.A'>


In [49]:
a is None

True

In [50]:
class A:
    def __new__(cls):
        print(f"__new__ {cls}")
        return object.__new__(cls)
    def __init__(self):
        print("__init__")

In [51]:
a = A()

__new__ <class '__main__.A'>
__init__


In [52]:
class A:
    def __new__(cls):
        print(f"__new__ {cls}")
        return super().__new__(cls)
    def __init__(self):
        print("__init__")

In [53]:
class A:
    def __new__(cls):
        print(f"__new__ {cls}")
        inst = super().__new__(cls)
        print(inst, type(inst))
        return inst
    def __init__(self):
        print("__init__")

In [54]:
a = A()

__new__ <class '__main__.A'>
<__main__.A object at 0x7fa9880446d0> <class '__main__.A'>
__init__


In [55]:
type("abc") # builtin function

str

In [56]:
A = type("A", (), {}) # 

In [57]:
type(A)

type

In [58]:
A = type("A", (object,), {"x": 1}) 

In [59]:
A.x

1

In [60]:
import datetime

In [61]:
class A:
    """ A """
    def __new__(cls, *args, **kwargs):
        inst = super().__new__(A)
        inst.__doc__ = inst.__doc__ + f" -- instatiated at {datetime.datetime.now()}"
        return inst
    
    def __init__(self, name, location="world"):
        self.name = name
        self.location = location

In [62]:
a = A("abc")

In [63]:
help(a)

Help on A in module __main__:

<__main__.A object>
    A  -- instatiated at 2022-12-08 11:06:24.064457



### Comparisons

* `__cmp__` - now gone

rich comparison operators:
    
* eq, ne, lt, gt, le, ge

In [79]:
class A:
    def __eq__(self, other):
        print("__eq__")
        return False

In [81]:
a = A()

In [82]:
a == "a"

__eq__


False

In [83]:
a != "a"

__eq__


True

In [84]:
class A:
    def __ne__(self, other):
        print("__ne__")
        return False

In [85]:
a = A()

In [86]:
a != "a"

__ne__


False

In [87]:
a == "a"

False

In [91]:
import functools

In [96]:
@functools.total_ordering # would not override existing implementions
class Word(str):
    def __eq__(self, other):
        return str(self) == str(other)
    
    def __lt__(self, other):
        return len(self) < len(other)

In [93]:
Word("abc") == Word("def")

False

In [94]:
Word("bb") < Word("aaa") # Word lt

True

In [95]:
Word("bb") > Word("aaa") # str gt

True

### Unary operator and function

Magic methods: pos, neg, abs, invert -- round, floor, ceil, trunc 





In [101]:
class A:
    def __abs__(self):
        return 0

In [102]:
a = A()

In [103]:
abs(a)

0

In [104]:
class A:
    def __invert__(self):
        return "inverted"

In [105]:
a = A()

In [106]:
~a

'inverted'

In [107]:
import math

In [109]:
class A:
    def __trunc__(self):
        return 1

In [110]:
a = A()

In [111]:
math.trunc(a)

1

### Augement assignment

* +=, -=, ...
* iadd, isub, imul, idiv, ...

In [117]:
class Votes:
    
    def __iadd__(self, other):
        self.votes = self.votes + other
        return self
        
    def __init__(self):
        self.votes = 0

In [118]:
v = Votes()

In [119]:
v += 1

In [120]:
v.votes

1

In [121]:
v += 1

In [122]:
v.votes

2

### Type conversion

* int, float, complex, oct, hex, ...

### String Magic (other stuff)



* str, repr, format
* hash, dir, bytes

* str vs repr

In [125]:
class A:
    
    def __repr__(self):
        return "repr A"

In [126]:
a = A()

In [127]:
a

repr A

In [129]:
print(a) # at this point __str__ == __repr__

repr A


In [134]:
class A:

    def __str__(self):
        return "str A"
    
    def __repr__(self):
        return "repr A"

In [136]:
a = A()

In [139]:
a

repr A

In [138]:
print(a)

str A


In [140]:
# what repr tries to be: eval(repr(a)) == a

In [141]:
print("%r" % a)

repr A


In [142]:
class A:

    def __str__(self):
        return 123

In [143]:
a = A()

In [146]:
# print(a) # TypeError: __str__ returned non-string (type int)

In [147]:
class A:
    def __bytes__(self):
        return b"abc"

In [148]:
a = A()

In [149]:
bytes(a)

b'abc'

In [150]:
class A:
    def __format__(self, format_spec):
        return f"A seen through the lens of '{format_spec}'"

In [151]:
a = A()

In [152]:
format(a, "%d")

"A seen through the lens of '%d'"

In [153]:
format(a, "{:03d}")

"A seen through the lens of '{:03d}'"

Hash special method

In [154]:
hash(1)

1

In [155]:
hash("hello")

6065124053262953902

In [156]:
x = 123
y = 123

In [157]:
hash(x) == hash(y)

True

In [161]:
id(x) == id(y) 

True

In [162]:
a = 1000000
b = 1000000

In [163]:
hash(a) == hash(b)

True

In [164]:
id(a) == id(b)

False

In [165]:
### Dir function

In [167]:
class A:
    def __dir__(self):
        return list("abcd")

In [168]:
a = A()

In [169]:
dir(a)

['a', 'b', 'c', 'd']

Sidenote

In [ ]:
`__sizeof__`

### Attribute magic

* getattr, setattr, delattr, getattribute

In [171]:
getattr(functools, "total_ordering")

<function functools.total_ordering(cls)>

In [172]:
setattr?

In [173]:
class A:
    def __getattr__(self, name):
        print("__getattr__ {name}")

In [174]:
a = A()

In [175]:
a.x

__getattr__ {name}


In [180]:
import re

In [181]:
class A:
    def __getattr__(self, name):
        m = re.match("(.*)_(.*)", name)
        if not m:
            raise AttributeError()
        match m.groups():
            case ["export", fmt]:
                match fmt:
                    case "pdf":
                        print("exp pdf")
                        return
                    case "png":
                        print("exp png")
                        return
        raise AttributeError()


In [182]:
a = A()

In [183]:
a.export_pdf

exp pdf


In [184]:
a.export_png

exp png


In [185]:
a.export_doc

AttributeError: 

* getattribute

In [187]:
class A:
    def __getattr__(self, name):
        print("__getattr__")
        return "dummy"
    
    def __getattribute__(self, name):
        print(f"__getattribute__ {name}")
        return "dummy"

In [188]:
a = A()

In [189]:
a.x

__getattribute__ x


'dummy'

In [192]:
class A:
    def __delattr__(self, name):
        print("__delattr__")

In [193]:
a = A()

In [194]:
a.x = 1

In [195]:
del a.x

__delattr__


### Operator magic

* add, sub, mul, floordiv, truediv, mod, pow

In [196]:
class A:
    def __add__(self, other):
        return "__add__"

In [197]:
a = A()

In [198]:
a + a

'__add__'

In [202]:
class Path:
    def __truediv__(self, other):
        self.segments.append(other)
        return self
    
    def __repr__(self):
        return "/" + "/".join(self.segments)
    
    def __init__(self):
        self.segments = []

In [203]:
p = Path()

In [204]:
p / "etc" / "passwd"

/etc/passwd

In [207]:
a - b # julia offers overloading of arbitrary unicode symbol

TypeError: unsupported operand type(s) for -: 'A' and 'int'

### Item access

* len, length_hint, getitem, setitem, delitem
* iter, next, reversed, contains

In [209]:
class A:
    def __len__(self):
        return 10

In [210]:
a = A()

In [211]:
len(a)

10

In [214]:
class Game:
    
    def __init__(self, w=5, h=5):
        self.w = w
        self.h = h
        self.grid = [i for i in range(w * h)]
    
    def __getitem__(self, key):
        if len(key) != 2:
            raise ValueError("tuple required")
        row, col = key
        return self.grid[col + self.w * row]

In [215]:
g = Game()

In [216]:
g.h

5

In [217]:
g.w

5

In [218]:
len(g.grid)

25

In [223]:
g[1, 1]

6

In [224]:
for i in range(g.h):
    for j in range(g.w):
        print("{: 4d}".format(g[i, j]), end="")
    print()

   0   1   2   3   4
   5   6   7   8   9
  10  11  12  13  14
  15  16  17  18  19
  20  21  22  23  24


### Descriptors

In [225]:
class A:
    v = 123

In [226]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'v': 123,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

* descriptor protocol: one of get, set, delete

In [228]:
class A:
    x = 10

In [229]:
a = A()

In [230]:
a.x

10

In [231]:
class D:
    def __get__(self, obj, objtype=None):
        print("D.__get__", obj)
        return 10

In [232]:
class A:
    x = D()

In [233]:
a = A()

In [234]:
a.x

D.__get__ <__main__.A object at 0x7fa963e65650>


10

In [235]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'x': <__main__.D at 0x7fa963e65a90>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [236]:
import os

In [239]:
class DirectorySize:
    
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

class Directory:
    
    size = DirectorySize()
    
    def __init__(self, dirname):
        self.dirname = dirname
        

In [240]:
d = Directory(".")

In [241]:
d.size

3

In [243]:
class Directory:
    
    def __init__(self, dirname):
        self.dirname = dirname
        
    @property
    def size(self):
        return self._compute_size()
    
    def _compute_size(self):
        return len(os.listdir(self.dirname)) 
    

In [244]:
d = Directory(".")

In [245]:
d.size

3

Example: logging, instance data

In [252]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:
    
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info("accessing %r giving %r", "age", value)
        return value
    
    def __set__(self, obj, value):
        logging.info("updating %r to %r", "age", value)
        obj._age = value

class Person:
    
    age = LoggedAgeAccess()
    
    def __init__(self, name, age):
        self.name = name 
        self.age = age
    
    def birthday(self):
        self.age += 1
        
        

In [253]:
mary = Person("mary", 30)

INFO:root:updating 'age' to 30


In [254]:
dave = Person("dave", 35)

INFO:root:updating 'age' to 35


In [255]:
mary.birthday()

INFO:root:accessing 'age' giving 30
INFO:root:updating 'age' to 31
